In [12]:
import cv2
import youtube_dl

TEXT_PATH = './data/data.txt'
SOURCE_PATH = './data/T1 vs DWG 1세트_144p.mp4' # https://tv.naver.com/v/12204239
HIGHLIGHT_URL = 'https://tv.naver.com/v/12203505'

class Video_capture:
    def __init__(self, video_path):
        self.cap = cv2.VideoCapture(video_path)
        self.ret = None 
        self.frame = None 
        
    def move_frame(self, frame_cnt):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_cnt)
        self.read()
        
    def make_grayscale(self):
        return cv2.cvtColor(self.frame, cv2.COLOR_BGR2GRAY)
    
    def read(self):
        self.ret, self.frame = self.cap.read()
        return

class Highlight_finder:
    def __init__(self, source, highlight):
        self.source = Video_capture(source)
        self.highlight= Video_capture(highlight)
        self.frame_cnt = 0
        self.highlight_frame_cnt = 210
        self.result = []
    
    def load_video(self):
        if not self.source.cap.isOpened():
                print('source video not opened')
                exit(-1)
        if not self.highlight.cap.isOpened():
                print('highlight video not opened')
                exit(-1)
        self.source.move_frame(self.frame_cnt)
        self.highlight.move_frame(self.highlight_frame_cnt)       
        return
    
    def match_template(self, src, frame):
        R = cv2.matchTemplate(src, frame, cv2.TM_CCORR_NORMED)
        _, maxVal, _, maxLoc = cv2.minMaxLoc(R)
        return maxVal
    
    def find_highlight(self):
        start = 0
        end = 0
        f = open("./data/" + SOURCE_PATH.split('/')[-1][:-4] +".txt",'w')
        while True: # 영상이 끝날때까지
            self.frame_cnt += 5
            if self.frame_cnt % 1800 == 0:
                print(self.frame_cnt / 1800, "min", self.highlight_frame_cnt)
            for i in range(5):
                self.source.read()
            if not self.source.ret or not self.highlight.ret:
                break
        #---------------------------------------------------------------------
            maxVal = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())

            #cv2.imshow('source', self.source.frame)
            key = cv2.waitKey(25)
            if key == 27:
                cv2.destroyAllWindows()
                break

            if maxVal < 0.95: # 영상이 일치하는지 체크
                continue
            else: # 일치한다면
                result = maxVal
                if(self.frame_cnt - end > 150):
                    if end != 0 and end - start >= 60:
                        f.write(str(start) + "," + str(end)+"\n")
                    start = self.frame_cnt
                while True: # 일치하지 않는 구간까지 반복
                    print(self.frame_cnt, result, self.highlight_frame_cnt)
                    self.highlight_frame_cnt += 15
                    self.frame_cnt += 15
                    for i in range(15):
                        self.highlight.read()
                        self.source.read()
                        
                    '''
                    cv2.imshow('source', self.source.frame)
                    cv2.imshow('highlight', self.highlight.frame)
                    key = cv2.waitKey(25)
                    if key == 27:
                        cv2.destroyAllWindows()
                    '''
                    if not self.source.ret or not self.highlight.ret: 
                        break
                    result = self.match_template(self.source.make_grayscale(), self.highlight.make_grayscale())
                    if result > 0.95: # 일치한다면
                        continue
                    else: # 일치하지 않는다면
                        print(self.frame_cnt, result, self.highlight_frame_cnt)
                        end = self.frame_cnt - 15
                        self.highlight_frame_cnt += 30
                        for i in range(30):
                            self.highlight.read()
                        break
                
        #----------------------------------------------------------------------
        if self.source.cap.isOpened():
            self.source.cap.release()

        if self.highlight.cap.isOpened():
            self.highlight.cap.release()
            
        f.write(str(start) + "," + str(end))
        f.close()
        #cv2.destroyAllWindows()
        print("END")
        return

In [ ]:
with open("./data/data.txt", "r") as src:
    while(True):
        source_path = src.readline()
        if not source_path : break
        source_path = "./data/" + source_path.split(',')[1].strip()
        highlight_path = source_path + "_HL.mp4"
        source_path += ".mp4"
        print(source_path)
        finder = Highlight_finder(source_path, highlight_path)
        finder.load_video()
        finder.find_highlight()

./data/20200411_APK_HLE_1.mp4
1.0 min 210
2.0 min 210
3.0 min 210
4.0 min 210
5.0 min 210
6.0 min 210
7.0 min 210
8.0 min 210
9.0 min 210
10.0 min 210
11.0 min 210
12.0 min 210
13.0 min 210
14.0 min 210
26100 0.9992541074752808 210
26115 0.9995412230491638 225
26130 0.6095271110534668 240
15.0 min 270
27135 0.9998067021369934 270
27150 0.9998586177825928 285
27165 0.9998430609703064 300
27180 0.9858434796333313 315
27195 0.8087942004203796 330
27220 0.950735330581665 360
27235 0.999923586845398 375
27250 0.9999533891677856 390
27265 0.9999532103538513 405
27280 0.7956535816192627 420
27315 0.9626916646957397 450
27330 0.9766311049461365 465
27345 0.9673756957054138 480
27360 0.8862839937210083 495
16.0 min 525
29920 0.9530746936798096 525
29935 0.9590139389038086 540
29950 0.9727398753166199 555
29965 0.9844117760658264 570
29980 0.9502577185630798 585
29995 0.9476348161697388 600
30025 0.9519693851470947 630
30040 0.9594108462333679 645
30055 0.956134021282196 660
30070 0.957538604736